In [2]:
import os

%pwd

'f:\\MLOps_Data\\Section-18-First-DataSciencd-Project-End-to-End\\datascienceproject\\research'

In [3]:
os.chdir("../")
%pwd

'f:\\MLOps_Data\\Section-18-First-DataSciencd-Project-End-to-End\\datascienceproject'

In [19]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path
    

In [20]:
from src.datascience.contants import *
from src.datascience.utils.common import read_yaml, create_directories

In [21]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)
        self.schema=read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_data_ingestion_config(self)-> DataIngestionconfig:
        config=self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config=DataIngestionconfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir

        )
        return data_ingestion_config

In [22]:
import os
import urllib.request as requests
from src.datascience import logger
import zipfile


In [23]:
## component Data Ingestion

class DataIngestion:
    def __init__(self,config:DataIngestionconfig):
        self.config=config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = requests.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists.")

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
            logger.info(f"Extracted zip file to {unzip_path}")
            


In [25]:
import os
os.chdir("f:/MLOps_Data/Section-18-First-DataSciencd-Project-End-to-End/datascienceproject")
print("Current working directory:", os.getcwd())

Current working directory: f:\MLOps_Data\Section-18-First-DataSciencd-Project-End-to-End\datascienceproject


In [26]:
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e


[2025-06-21 23:15:28,330: INFO: common: yaml file: config\config.yaml loaded successfully.]
[2025-06-21 23:15:28,340: INFO: common: yaml file: params.yaml loaded successfully.]
[2025-06-21 23:15:28,344: INFO: common: yaml file: schema.yaml loaded successfully.]
[2025-06-21 23:15:28,347: INFO: common: created directory at: artifacts]
[2025-06-21 23:15:28,349: INFO: common: created directory at: artifacts/data_ingestion]
[2025-06-21 23:15:29,640: INFO: 2877490283: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 376D:3CC1B2:12FFFE:395165:6856EFBA
Accept-Ranges: bytes
Date: Sat,